In [1]:
import pandas as pd
import numpy as np
import ast 
import unicodedata
import pickle

from sklearn.metrics import precision_recall_fscore_support

In [2]:
documents = pd.read_csv("../../files/documents.csv", )
documents.head()

,id,title,content,category,date
0,0,Auditoría revela irregularidades en el Parlacen,GUATEMALA.- Una fiscalización de la Contralorí...,Other,"28 Dic, 2009 - 7:27 pm"
1,1,Suspendidas las citas en Hospital Escuela,TEGUCIGALPA.- Una misteriosa obstrucción del s...,Other,"28 Dic, 2009 - 7:32 pm"
2,2,Mariscos contaminados alarman a los “porteños”,"PUERTO CORTES, Cortés.- Alarmados se encuentra...",Other,"28 Dic, 2009 - 8:25 pm"
3,3,Citan a 11 personas por vender pólvora,SAN PEDRO SULA.- Hasta el momento ocho bodegas...,Criminal,"28 Dic, 2009 - 8:26 pm"
4,4,Con compra de granos se paliaría hambruna en e...,TEGUCIGALPA.- No llueve hace cuatro meses y la...,Other,"29 Dic, 2009 - 1:00 am"


In [3]:
# pre-process (clean the data)
documents = documents[(documents.content.notnull() ) 
                            & (documents["category"]!="Criminal-Other")]

In [4]:
# Split
train_documents = documents.loc[:1999]
test_documents = documents.loc[2000:]

In [171]:
test_documents[ (test_documents["category"] != "Criminal") & (test_documents["pred"] == 1)]

,id,title,content,category,date,pred
2023,2023,Muere hombre al chocar contra una plantación d...,"San José, (ACAN-EFE).- Un hombre falleció al e...",Other,"27 Mar, 2015 - 2:04 pm",1
2029,2029,Se reporta primer ahogado previo a inicio de S...,"El primer ahogado en vísperas de Semana Santa,...",Other,"27 Mar, 2015 - 3:20 pm",1
2038,2038,Incineran marihuana en basurero de Comayagua,Comayagua. Autoridades policiales incineraron ...,Other,"27 Mar, 2015 - 6:30 pm",1
2049,2049,Llega a Honduras violador en serie (VIDEO),"El violador serial hondureño, Hugo Edgardo Sie...",Other,"27 Mar, 2015 - 7:34 pm",1
2088,2088,Madre de colegiala ultimada: “Les pido no poli...,"Ceila Bustillo, madre de Soad Nicole Ham Busti...",Other,"28 Mar, 2015 - 1:12 am",1
2118,2118,Familiares de internos en Penitenciaría Nacion...,Familiares de los internos en la Penitenciaría...,Other,"28 Mar, 2015 - 10:44 am",1
2123,2123,Cuatro personas heridas en accidente automovil...,Cuatro personas heridas fue el resultado dejad...,Other,"28 Mar, 2015 - 12:58 pm",1
2126,2126,Fusina y PMOP se toman Penitenciaría Nacional ...,Elementos de la Fuerza de Seguridad Interinsti...,Other,"28 Mar, 2015 - 1:57 pm",1
2137,2137,Menor de 13 años de edad ahogado en occidente ...,"Un menor de 13 años de edad, murió ahogado hoy...",Other,"28 Mar, 2015 - 3:20 pm",1
2145,2145,Muere empresario nicaragüense al sufrir accide...,Un empresario nicaragüense identificado como R...,Other,"28 Mar, 2015 - 8:04 pm",1


# TOPIC

In [5]:
topic_clf = pickle.load( open( "../../models/topic_model.p", "rb" ) )

In [154]:
print topic_clf.alpha
topic_clf.content

[0.5]


{'analyzer': 'word',
 'binary': True,
 'max': 0.9,
 'min': 1,
 'ngram': [1, 1],
 'vectorizer': 'count'}

In [6]:
train_preds = topic_clf.predict(train_documents)
test_preds = topic_clf.predict(test_documents)

train_true = train_documents.category.apply(lambda x: int(x=="Criminal"))
test_true = test_documents.category.apply(lambda x: int(x=="Criminal"))

In [7]:
print len(train_preds)
print len(train_true)
print len(test_preds)
print len(test_true)

train_documents.loc[:,"pred"] = train_preds
test_documents.loc[:,"pred"] = test_preds

1710
1710
479
479


/Users/alextabora/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/alextabora/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:476: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [8]:
print precision_recall_fscore_support(train_true,train_preds, average="binary")
print precision_recall_fscore_support(test_true, test_preds, average="binary")

(0.876, 0.9776785714285714, 0.92405063291139233, None)
(0.68478260869565222, 0.92647058823529416, 0.78750000000000009, None)


# POS TAG

In [9]:
from nltk.tokenize import wordpunct_tokenize, sent_tokenize

def tokenize(base_df):
    data = []
    for art_index, article in base_df.iterrows():
        sentences = sent_tokenize(unicode(article.content,"utf8"))
        sentences = [wordpunct_tokenize(sent) for sent in sentences ]
        for index in range(len(sentences)):
            sentences[index] = [ [word,"none"] for word in sentences[index]]
        # Title
        title = wordpunct_tokenize(unicode(article.title,"utf8"))
        title = [ [word,"none"] for word in title ]

        #append
        data.append([
            art_index,
            title,
            sentences
        ])
        
    return pd.DataFrame(data,columns=["article_id","title","content"])

In [10]:
filtered_train = train_documents[train_documents["pred"]==1].copy()
filtered_test = test_documents[test_documents["pred"]==1].copy()

filtered_train = tokenize(filtered_train)
filtered_test = tokenize(filtered_test)

In [174]:
len(filtered_train.loc[0].content)  ##ONLY 2 sentences for the art_id == 3 !!! :O  :O : O

2

In [11]:
tagger = pickle.load( open( "../../files/pos_tagger.p", "rb" ) )

def tagContent(content):
        for index, sentence in enumerate(content):
            words = [ word[0] for word in sentence]
            #tag
            pos_sentence = tagger.tag(words)
            #add tag to sentence
            content[index] = [ [word[0],pos_tag[1], word[1]] for word, pos_tag in zip(sentence, pos_sentence) ]
        return content
    
def tagTitle(title):
        sentence = [ word[0] for word in title]
        pos_sentence = tagger.tag(sentence)

        tagged_title = [ [word[0],pos_tag[1],word[1]] for word, pos_tag in zip(title, pos_sentence) ]

        return tagged_title
      
# pos tagging
print "tagging train"
filtered_train.loc[:,"tagged_title"] = filtered_train.title.apply(lambda title: tagTitle(title))
filtered_train.loc[:,"tagged_content"] = filtered_train.content.apply(lambda content:  tagContent(content) )

print "tagging test"
filtered_test.loc[:,"tagged_title"] = filtered_test.title.apply(lambda title: tagTitle(title))
filtered_test.loc[:,"tagged_content"] = filtered_test.content.apply(lambda content:  tagContent(content) )

print "all datasets tagged"

tagging train
tagging test
all datasets tagged


In [12]:
filtered_train.head()

,article_id,title,content,tagged_title,tagged_content
0,3,"[[Citan, none], [a, none], [11, none], [person...","[[[SAN, AQ, none], [PEDRO, NC, none], [SULA, N...","[[Citan, NC, none], [a, SP, none], [11, Z, non...","[[[SAN, AQ, none], [PEDRO, NC, none], [SULA, N..."
1,5,"[[DEI, none], [pide, none], [denunciar, none],...","[[[TEGUCIGALPA, NC, none], [.-, NC, none], [Au...","[[DEI, NC, none], [pide, VMI, none], [denuncia...","[[[TEGUCIGALPA, NC, none], [.-, NC, none], [Au..."
2,9,"[[Alcaldía, none], [intensifica, none], [opera...","[[[TEGUCIGALPA, NC, none], [.-, NC, none], [Pe...","[[Alcaldía, NC, none], [intensifica, NC, none]...","[[[TEGUCIGALPA, NC, none], [.-, NC, none], [Pe..."
3,11,"[[Pasajeros, none], [asaltantes, none], [acrib...","[[[PUERTO, NC, none], [CORTES, NC, none], [,, ...","[[Pasajeros, NC, none], [asaltantes, NC, none]...","[[[PUERTO, NC, none], [CORTES, NC, none], [,, ..."
4,12,"[[Fallece, none], [comerciante, none], [olanch...","[[[JUTICALPA, NC, none], [,, Fc, none], [Olanc...","[[Fallece, NC, none], [comerciante, NC, none],...","[[[JUTICALPA, NC, none], [,, Fc, none], [Olanc..."


# NER

In [13]:
import sys
import imp

import location.preprocess as process
imp.reload(sys.modules['location.preprocess'])

<module 'location.preprocess' from 'location/preprocess.pyc'>

In [14]:
sentences_train = process.convertToSentences(filtered_train, toObject=False)
sentences_test = process.convertToSentences(filtered_test, toObject=False)

- Convert to Sentences
- Convert to Sentences


In [15]:
words_train = process.sentenecesToWords(sentences_train)
words_test = process.sentenecesToWords(sentences_test)

words_train = words_train[words_train["word"] != "."]
words_test = words_test[words_test["word"] != "."]

- Sentences to Words
- Sentences to Words


In [16]:
words_train.head(10)

,art_id,sent_id,cs_id,pos,word,pos_tag,iob_tag,iob,geo_type
0,3,0,0,0,SAN,AQ,none,O,none
1,3,0,0,1,PEDRO,NC,none,O,none
2,3,0,0,2,SULA,NC,none,O,none
3,3,0,0,3,.-,NC,none,O,none
4,3,0,0,4,Hasta,SP,none,O,none
5,3,0,0,5,el,DA,none,O,none
6,3,0,0,6,momento,NC,none,O,none
7,3,0,0,7,ocho,DN,none,O,none
8,3,0,0,8,bodegas,NC,none,O,none
9,3,0,0,9,han,VAI,none,O,none


In [36]:
final_model = pickle.load( open( "../../models/NER_model.p", "rb" ) )
final_model

{'le_iob': LabelEncoder(),
 'le_tag': LabelEncoder(),
 'lv1': [<location.classifiers.Classifier instance at 0x1115aa560>,
  <location.classifiers.Classifier instance at 0x112a445a8>,
  <location.classifiers.Classifier instance at 0x112a44d40>],
 'lv2': <location.classifiers.Classifier instance at 0x112a49ab8>,
 'lv3': <location.classifiers.Classifier instance at 0x11110bb90>}

In [18]:
features_train, _, _ = process.getFeatures(
    words_train.copy(), le_tag= final_model["le_tag"], le_iob= final_model["le_iob"], root_folder="../../")
features_test, _, _ = process.getFeatures(
    words_test.copy(), le_tag= final_model["le_tag"], le_iob= final_model["le_iob"], root_folder="../../")

- getting features
-- iob tag classes
['B' 'O']
- getting features
-- iob tag classes
['B' 'O']


In [19]:
features_test.isnull().any()

art_id                 False
sent_id                False
cs_id                  False
pos                    False
word                   False
pos_tag                False
iob_tag                False
iob                    False
geo_type               False
upper                  False
upper_prev1            False
upper_next1            False
first                  False
first_sent             False
size                   False
prev_1                 False
prev_2                 False
next_1                 False
next_2                 False
triggerstate1          False
triggerstate2          False
triggercity1           False
triggercity2           False
triggerzone1           False
triggerzone2           False
triggercolonia1        False
triggercolonia2        False
triggerbarrio1         False
triggerbarrio2         False
triggerresidencial1    False
triggerresidencial2    False
trigger_1              False
trigger_2              False
sent_size              False
in_Country    

In [20]:
desc_columns = ["art_id","sent_id", "cs_id","word", "iob_tag","iob" ,"geo_type"]
X_train = features_train.drop(desc_columns, 1).values
X_test = features_test.drop(desc_columns, 1).values

#lvl 1 classifier
stack_features_train = features_train.copy()  #[desc_columns]
stack_features_test = features_test.copy() #[desc_columns]
for idx, clf in enumerate(final_model["lv1"]):
    preds_train = clf.predict(X_train)
    preds_test = clf.predict(X_test)
    
    stack_features_train.loc[:,"pred_"+str(idx)] = preds_train
    stack_features_test.loc[:,"pred_"+str(idx)] = preds_test

## LV2 classifier

In [26]:
import location.identifier as identifier
imp.reload(sys.modules['location.identifier'])

<module 'location.identifier' from 'location/identifier.pyc'>

In [22]:
stack_features_train = identifier.get_stacked_features(stack_features_train, final_model["lv1"])
stack_features_test = identifier.get_stacked_features(stack_features_test, final_model["lv1"])

X_train = stack_features_train.drop(desc_columns, 1).values
X_test = stack_features_test.drop(desc_columns, 1).values

In [23]:
preds_train = final_model["lv2"].predict(X_train)
preds_test = final_model["lv2"].predict(X_test)

In [25]:
preds_train[:40]

array([0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1])

## LV3 classifier

In [37]:
# . Prepare to classify JUST the entities
stack_features_train["s_pred"] = preds_train
stack_features_test["s_pred"] = preds_test

cat_train = identifier.getOnlyB(stack_features_train)
cat_test = identifier.getOnlyB(stack_features_test)

X_train = cat_train.drop(desc_columns, 1).values
y_train = cat_train.iob_tag.apply(lambda x: x[2:]).values

X_test = cat_test.drop(desc_columns, 1).values
y_test = cat_test.iob_tag.apply(lambda x: x[2:]).values

In [38]:
final_preds_train = final_model["lv3"].predict(X_train)
final_preds_test = final_model["lv3"].predict(X_test)

In [39]:
final_preds_train

array([u'City', u'City', u'City', ..., u'City', u'Zone', u'Zone'], dtype=object)

## Extract entities

In [30]:
from  location.entities import extract_entities, score_entities
imp.reload(sys.modules['location.entities'])

<module 'location.entities' from 'location/entities.pyc'>

In [40]:
geo_preds_train = pd.Series(final_preds_train, index = cat_train.index )
geo_preds_test = pd.Series(final_preds_test, index = cat_test.index )

final_train = stack_features_train.copy()
final_test = stack_features_test.copy()

final_train.loc[:,"pred_geo"] = geo_preds_train
final_test.loc[:,"pred_geo"] = geo_preds_test

# --Doesn't Apply--
# true_ent_train = extract_entities(features_train,"iob", "geo_type")
# true_ent_test = extract_entities(features_test ,"iob", "geo_type")

train_entities = extract_entities(final_train,"s_pred", "pred_geo")
test_entities = extract_entities(final_test,"s_pred", "pred_geo")

In [165]:
test_entities[50:100]

,art_id,sent_id,cs_id,pos,entity,geo_type
50,2049,0,12,144,honduras,Country
51,2049,0,12,182,tegucigalpa,City
52,2049,0,12,224,san pedro sula,City
53,2049,0,12,263,tegucigalpa,City
54,2049,0,12,301,tamara,City
55,2049,0,12,320,honduras,Country
56,2049,0,12,356,tapachula,City
57,2049,0,12,361,chiapas,State
58,2049,0,12,363,mexico,Country
59,2049,0,12,412,mexico,Country


# RELATIONSHIPS

In [43]:
def to_ascii(s):
    return unicodedata.normalize('NFKD', s).encode('ascii', 'ignore')

train_entities.loc[:,"entity"] = train_entities.entity.apply(lambda x: to_ascii(unicode(x).lower() ) )
test_entities.loc[:,"entity"] = test_entities.entity.apply(lambda x: to_ascii(unicode(x).lower()) )

In [47]:
final_preds_train

array([u'City', u'City', u'City', ..., u'City', u'Zone', u'Zone'], dtype=object)

In [65]:
def add_results_to_words(words_df):
    new_words = words_df[["art_id","sent_id","cs_id","pos","word", "s_pred"]].copy()
    new_words.loc[:,"word"] = new_words["word"].apply(lambda x: to_ascii( unicode(x).lower() ) )
    new_words.loc[:,"iob_tag"] = words_df.pred_geo.apply(lambda x: x if not pd.isnull(x) else "Other")
    new_words.loc[:,"iob"] = new_words.iob_tag.apply(lambda x: "B" if x!="Other" else "O")
    #correct by adding the respective I
    last_iob = "O"
    for idx, row in new_words.iterrows():
        if row.iob == "O" and row.s_pred == 0 and ( last_iob == "B" or last_iob == "I" ):
            new_words.loc[idx,"iob"] = "I"
        last_iob = new_words.loc[idx].iob
    

    return new_words
#tagged words
twords_train = add_results_to_words(final_train)
twords_test = add_results_to_words(final_test)

twords_train.head()  #[(twords_train["art_id"]==3)&(twords_train["iob"]=="B")]

,art_id,sent_id,cs_id,pos,word,s_pred,iob_tag,iob
0,3,0,0,0,san,0,City,B
1,3,0,0,1,pedro,0,Other,I
2,3,0,0,2,sula,0,Other,I
3,3,0,0,3,.-,1,Other,O
4,3,0,0,4,hasta,1,Other,O


In [56]:
twords_test

,art_id,sent_id,cs_id,pos,word,s_pred,iob_tag,iob
0,2023,0,0,0,San,0,City,B
1,2023,0,0,1,José,0,Other,I
2,2023,0,0,2,",",1,Other,O
3,2023,0,0,3,(,1,Other,O
4,2023,0,0,4,ACAN,1,Other,O
5,2023,0,0,5,-,1,Other,O
6,2023,0,0,6,EFE,1,Other,O
7,2023,0,0,7,).-,1,Other,O
8,2023,0,0,8,Un,1,Other,O
9,2023,0,0,9,hombre,1,Other,O


In [66]:
def get_distance_df(df):
    
    artid = -1
    data = []
    word_count = 0
    for _, word in df.iterrows():
        if word.iob == "B":
            data.append([word.word.lower(), word.iob_tag, word.sent_id, word.pos, 0])
        elif word.iob == "I":
            data[-1][0] += " " + word.word.lower()
        elif len(data) > 0:
            data[-1][-1] += 1
            
    entity_df = pd.DataFrame(data, columns=["entity", "tag", "sent_id", "pos", "next_words"])
    return entity_df 
            
            
get_distance_df(twords_train[twords_train["art_id"]==3])         

,entity,tag,sent_id,pos,next_words
0,san pedro sula,City,0,0,26
1,san pedro sula,City,0,29,16
2,san pedro sula,City,0,49,38
3,lomas,Col,0,91,1
4,carmen,City,0,93,58
5,madre,Zone,0,153,47
6,san pedro sula,City,1,26,48
7,del medio,Col,1,78,14


In [76]:
def getPairs(df):
    art_id = df.values[0][0]
    
    country_entities = df[df["geo_type"]=="Country"][["geo_type","entity"]].values.tolist()
    state_entities = df[df["geo_type"]=="State"][["geo_type","entity"]].values.tolist()
    city_entities = df[df["geo_type"]=="City"][["geo_type","entity"]].values.tolist()
    zone_entities = df[df["geo_type"].isin(["Zone","Col","Bar","Res"])][["geo_type","entity"]].values.tolist()

    pairs = []
    for child in zone_entities:
        for parent in city_entities + state_entities + country_entities:
            pairs.append([art_id,child[0], child[1].lower(), parent[0], parent[1].lower(), 
                         {"word":child[1],"tag":child[0]}])

    for child in city_entities:
        for parent in  state_entities + country_entities:
            pairs.append([art_id,child[0], child[1].lower(), parent[0], parent[1].lower(), 
                          {"word":child[1],"tag":child[0]}])

    for child in state_entities:
        for parent in country_entities:
            pairs.append([art_id,child[0], child[1].lower(), parent[0], parent[1].lower(), 
                         {"word":child[1],"tag":child[0]}])
      
    relations_df = pd.DataFrame(pairs,columns=["art_id","c_tag","c_word","p_tag","p_word","child"])
    
    relations_df.loc[:,"rel_type"] = relations_df.c_tag +["-" for x in range(relations_df.shape[0])] +relations_df.p_tag 
    return relations_df

getPairs(train_entities[train_entities["art_id"]==3].drop_duplicates(subset=["entity"]) ) 

,art_id,c_tag,c_word,p_tag,p_word,child,rel_type
0,3,Col,lomas,City,san pedro sula,"{u'tag': u'Col', u'word': u'lomas'}",Col-City
1,3,Col,lomas,City,carmen,"{u'tag': u'Col', u'word': u'lomas'}",Col-City
2,3,Zone,madre,City,san pedro sula,"{u'tag': u'Zone', u'word': u'madre'}",Zone-City
3,3,Zone,madre,City,carmen,"{u'tag': u'Zone', u'word': u'madre'}",Zone-City
4,3,Col,del medio,City,san pedro sula,"{u'tag': u'Col', u'word': u'del medio'}",Col-City
5,3,Col,del medio,City,carmen,"{u'tag': u'Col', u'word': u'del medio'}",Col-City


## get features

In [59]:
import relationships.preprocess as relationship

In [87]:
def preprocess_rel(words_df, entities_df):
    relationships = pd.DataFrame()
    for art_id in words_df.art_id.unique():
        try:
            if words_df[words_df["art_id"] == art_id].shape[0] == 0 or \
                entities_df[entities_df["art_id"] == art_id].shape[0] == 0:
                continue

    #         print "---- %i ----" %art_id
            # get entities distance in the article
            distance_df = get_distance_df( words_df[words_df["art_id"] == art_id] )
    #         print distance_df

            # generate possible relationship pairs
            relations_df = getPairs( entities_df[entities_df["art_id"] == art_id].drop_duplicates(subset=["entity"]) )
    #         print relations_df

            # generate features
            features_df = relationship.getFeatures(relations_df, distance_df)
            features_df.art_id = art_id

            # append the relationships to the final dataframe
            relationships = pd.concat([relationships,features_df])
        except Exception as error:
            print "exception in %i"  %art_id
            print error
            
            print distance_df
            print relations_df
            
            return None
            
    
    print "relationships size %i,%i"%(relationships.shape)
    return relationships

relationship_train = preprocess_rel(twords_train, train_entities)
relationship_train.head()

relationships size 2444,25


,art_id,c_tag,c_word,p_tag,p_word,child,rel_type,distances,minDistance,maxDistance,...,p_first_sent,c_title,p_title,p_torf,c_torf,min_all,num_rels,state_country,city_state,city_country
0,3,Col,lomas,City,san pedro sula,"{u'tag': u'Col', u'word': u'lomas'}",Col-City,"[82, 55, 38, 108]",38.0,108.0,...,1.0,1.0,1.0,True,True,1.0,2.0,0.0,0.0,0.0
1,3,Col,lomas,City,carmen,"{u'tag': u'Col', u'word': u'lomas'}",Col-City,[1],1.0,1.0,...,0.0,1.0,1.0,True,True,1.0,2.0,0.0,0.0,0.0
2,3,Zone,madre,City,san pedro sula,"{u'tag': u'Zone', u'word': u'madre'}",Zone-City,"[143, 116, 99, 47]",47.0,143.0,...,1.0,1.0,1.0,True,True,47.0,2.0,0.0,0.0,0.0
3,3,Zone,madre,City,carmen,"{u'tag': u'Zone', u'word': u'madre'}",Zone-City,[58],58.0,58.0,...,0.0,1.0,1.0,True,True,47.0,2.0,0.0,0.0,0.0
4,3,Col,del medio,City,san pedro sula,"{u'tag': u'Col', u'word': u'del medio'}",Col-City,"[240, 213, 196, 48]",48.0,240.0,...,1.0,0.0,1.0,True,True,48.0,2.0,0.0,0.0,0.0


In [88]:
relationship_test = preprocess_rel(twords_test, test_entities)
relationship_test.head()

relationships size 743,25


,art_id,c_tag,c_word,p_tag,p_word,child,rel_type,distances,minDistance,maxDistance,...,p_first_sent,c_title,p_title,p_torf,c_torf,min_all,num_rels,state_country,city_state,city_country
0,2023,Zone,el laurel,City,san jose,"{u'tag': u'Zone', u'word': u'el laurel'}",Zone-City,[68],68.0,68.0,...,0.0,1.0,1.0,True,True,16.0,3.0,0.0,0.0,0.0
1,2023,Zone,el laurel,Country,costa rica,"{u'tag': u'Zone', u'word': u'el laurel'}",Zone-Country,[31],31.0,31.0,...,0.0,1.0,1.0,True,True,16.0,3.0,0.0,0.0,0.0
2,2023,Zone,el laurel,Country,panama,"{u'tag': u'Zone', u'word': u'el laurel'}",Zone-Country,[16],16.0,16.0,...,0.0,1.0,1.0,True,True,16.0,3.0,0.0,0.0,0.0
3,2023,Zone,morgue,City,san jose,"{u'tag': u'Zone', u'word': u'morgue'}",Zone-City,[125],125.0,125.0,...,0.0,1.0,1.0,True,True,39.0,3.0,0.0,0.0,0.0
4,2023,Zone,morgue,Country,costa rica,"{u'tag': u'Zone', u'word': u'morgue'}",Zone-Country,[88],88.0,88.0,...,0.0,1.0,1.0,True,True,39.0,3.0,0.0,0.0,0.0


## Make relationship predictions

In [89]:
from sklearn.preprocessing import LabelEncoder
def get_X(df, desc_columns):
    le = LabelEncoder()
    #encode c_tag and p_tag
    df.loc[:,"c_tag"] = le.fit_transform(df.c_tag)
    df.loc[:,"p_tag"] = le.fit_transform(df.p_tag)
    df.loc[:,"rel_type"] = le.fit_transform(df.rel_type)
    return df.drop(desc_columns,1).values

In [90]:
desc_columns = [ 'art_id', 'c_word', 'p_word', 'child', 'distances']
X_train = get_X(relationship_train.copy(), desc_columns)
X_test = get_X(relationship_test.copy(), desc_columns)

In [92]:
#read the file
rel_model = pickle.load( open( "../../models/rel_model.p", "rb" ) )
# ----> AQUIII YA CASIIII <----
relpreds_train = rel_model["model"].predict(X_train)
relpreds_test = rel_model["model"].predict(X_test)

In [94]:
relpreds_train[:10]

array([1, 1, 1, 0, 1, 0, 1, 0, 0, 1])

# FINAL Phase
## first generate the addresses based on relationships

In [102]:
final_rel_train = relationship_train[["art_id","c_tag","c_word","p_tag","p_word","rel_type"]].copy()
final_rel_test = relationship_test[["art_id","c_tag","c_word","p_tag","p_word","rel_type"]].copy()

final_rel_train.loc[:,"pred"] = relpreds_train
final_rel_test.loc[:,"pred"] = relpreds_test

final_rel_test[final_rel_test["pred"]==1].head(40)

,art_id,c_tag,c_word,p_tag,p_word,rel_type,pred
0,2023,Zone,el laurel,City,san jose,Zone-City,1
3,2023,Zone,morgue,City,san jose,Zone-City,1
6,2023,City,san jose,Country,costa rica,City-Country,1
0,2027,City,jocon,State,yoro,City-State,1
6,2027,State,yoro,Country,honduras,State-Country,1
0,2029,City,santa rosa de copan,State,copan,City-State,1
4,2029,State,copan,Country,honduras,State-Country,1
0,2034,Zone,platano,City,tegucigalpa,Zone-City,1
6,2034,Zone,jose heriberto,City,rio abajo,Zone-City,1
10,2034,City,tegucigalpa,State,olancho,City-State,1


In [125]:
def searchParentRel(p_word, p_tag, article):
    address = "" 
    address_tags = "" 
    if len(article[p_tag]) == 0:
        return address, address_tags
    
    for idx, rel in enumerate(article[p_tag]):
        if rel[0].c_word == p_word:
            article[p_tag][idx][1] = True  #set to visited
            address = ", " + rel[0]["p_word"]
            address_tags = " - " +  rel[0]["p_tag"]
            p_address, p_address_tags = searchParentRel(rel[0].p_word, rel[0].p_tag, article)
            address += p_address
            address_tags += p_address_tags
            return address, address_tags
    return address, address_tags

def getAddressByRoot(geo_type,article, data):
    for rel_idx, rel in enumerate(article[geo_type]):
        if rel[1] == False:
            address = rel[0].c_word + ", " + rel[0].p_word
            address_tags = rel[0].c_tag + " - " + rel[0].p_tag
            #add grand-parent address
            p_address, p_address_tags = searchParentRel(rel[0].p_word, rel[0].p_tag, article)
            address += p_address
            address_tags += p_address_tags
            data.append([rel[0].art_id, address, address_tags])


def getAdresses(df):
    #group by articles
    art_childs = []
    last_artid = -1
    for _, rel in df[df["pred"]==1].iterrows():
        if rel.art_id != last_artid:
            art_childs.append({
                "Country":[],
                "State":[],
                "City":[],
                "Zone":[],
                "Col":[],
                "Bar":[],
                "Res":[] 
            })
        #the False is to know if it was visited
        art_childs[-1][rel.c_tag].append( [rel, False] )
        last_artid = rel.art_id 
            
    #
    data = []
    for art_idx, article in enumerate(art_childs):
        #first start with zones
        getAddressByRoot("Zone",article, data)
        getAddressByRoot("Col",article, data)
        getAddressByRoot("Bar",article, data)
        getAddressByRoot("Res",article, data)
        getAddressByRoot("City",article, data)
        getAddressByRoot("State",article, data)
        getAddressByRoot("Country",article, data)

    return pd.DataFrame(data,columns=["art_id","address", "address_tags"])
getAdresses(final_rel_test[final_rel_test["pred"]==1]).head(20) 

,art_id,address,address_tags
0,2023,"el laurel, san jose, costa rica",Zone - City - Country
1,2023,"morgue, san jose, costa rica",Zone - City - Country
2,2027,"jocon, yoro, honduras",City - State - Country
3,2029,"santa rosa de copan, copan, honduras",City - State - Country
4,2034,"platano, tegucigalpa, olancho",Zone - City - State
5,2034,"jose heriberto, rio abajo, olancho",Zone - City - State
6,2034,"santos, olancho",City - State
7,2034,"carnes, olancho",City - State
8,2038,"segun, comayagua",Zone - City
9,2039,"san pedro sula, honduras",City - Country


In [140]:
pred_addresses_train = getAdresses(final_rel_train[final_rel_train["pred"]==1])
pred_addresses_test = getAdresses(final_rel_test[final_rel_test["pred"]==1])

In [148]:
addresses_df = pd.read_csv("../../files/expected_addresses.csv", )
addresses_df.loc[:,"composition"] = addresses_df["shape"].apply(lambda x: len(x.split("-")) )
addresses_df.head()

,art_id,shape,address,incomplete,missing_country,missing_city,composition
0,3,Col-City,"lomas del carmen, san pedro sula",1,1,0,2
1,3,Zone-City,"monumento a la madre, san pedro sula",1,1,0,2
2,3,Zone-City,"primera avenida, san pedro sula",1,1,0,2
3,3,Zone-City,"parque central, san pedro sula",1,1,0,2
4,5,City,tegucigalpa,1,1,0,1


In [149]:
true_addresses_train = addresses_df[addresses_df["art_id"]<2000]
true_addresses_test = addresses_df[addresses_df["art_id"]>2000]

In [168]:
#Totally unneficient way but lets do it
def findAddress(art_id, address, df):
    return df[(df["art_id"]==art_id)&(df["address"]==address)].shape[0] > 0

def scoreAddresses(pred_df, true_df):
    tp =0
    fp = 0
    tp_df = []
    fp_df = []
    for _, address in pred_df.iterrows():
        if findAddress(address.art_id, address.address, true_df):
            tp +=1
            tp_df.append(address.values)
        else:
            fp +=1
            fp_df.append(address.values)
            
    support = true_df.shape[0]
    print "True positives: %i" %tp
    print "False positives: %i" %fp
    print "Precision: %0.4f" % (tp*1.0/(tp+fp))
    print "Recall: %0.4f" % (tp*1.0/support)
    print "Support: %i" %support
    
    
    tp_df = pd.DataFrame(tp_df, columns=["art_id","address", "address_tags"])
    fp_df = pd.DataFrame(fp_df, columns=["art_id","address", "address_tags"])
    
    return tp_df, fp_df
    
train_tp, train_fp = scoreAddresses(pred_addresses_train, true_addresses_train[true_addresses_train["composition"]>1])
print 
test_tp, test_fp = scoreAddresses(pred_addresses_test, true_addresses_test[true_addresses_test["composition"]>1])

True positives: 203
False positives: 368
Precision: 0.3555
Recall: 0.3988
Support: 509

True positives: 30
False positives: 131
Precision: 0.1863
Recall: 0.2655
Support: 113


In [134]:
pred_addresses_train.head(1)

,art_id,address,address_tags
0,2023,"el laurel, san jose, costa rica",Zone - City - Country


In [170]:
train_fp

,art_id,address,address_tags
0,3,"madre, san pedro sula",Zone - City
1,3,"lomas, san pedro sula",Col - City
2,3,"lomas, carmen",Col - City
3,3,"del medio, san pedro sula",Col - City
4,11,"lopez, puerto cortes, cortes",Col - City - State
5,12,"villanueva, honduras",City - Country
6,12,"san francisco, honduras",City - Country
7,13,"la, el negrito, yoro",Zone - City - State
8,13,"el negrito, yoro victoria",City - State
9,13,"victoria, yoro",City - State
